In [1]:
# %% [markdown]
# # Logic Field Theory Simulation: Paradox Filtering (GHZ & EPR)
#
# This notebook simulates how Logic Field Theory (LFT) handles quantum paradoxes like GHZ and correlations like EPR through its core mechanism: the Logical Projection Operator (`Π_L`). LFT posits that physically instantiated reality (`Ω`) only contains logically coherent information states (`s ∈ S`). `Π_L` filters out any potential state that entails a contradiction.

# %%
import itertools
import numpy as np

# %% [markdown]
# ## LFT Core Functions (Simplified for Paradox Scenarios)

# %%
def negate(prop):
  """Simple helper for proposition negation."""
  if prop.startswith("¬"):
    return prop[1:]
  else:
    return f"¬{prop}"

def is_consistent(state, rules):
  """
  Checks if a state (set of propositions) is logically consistent.
  Includes direct contradiction check and violation of specific scenario rules.
  Args:
    state (set): Set of proposition strings.
    rules (list): List of functions, each taking the state and returning True if
                  a specific rule is violated, False otherwise.
  Returns:
    bool: True if consistent, False otherwise.
  """
  # 1. Direct Contradiction Check
  for prop in state:
    if negate(prop) in state:
      # print(f"Direct contradiction found: {prop} and {negate(prop)}")
      return False

  # 2. Scenario-Specific Rule Check
  for rule_violated in rules:
    if rule_violated(state):
      # print(f"Rule violation detected by: {rule_violated.__name__}")
      return False

  return True

def Pi_L(state, rules):
  """
  The Logical Projection Operator (Π_L).
  Filters a state for logical coherence.
  Returns:
    set or None: The state if consistent, None otherwise (representing Ø).
  """
  if is_consistent(state, rules):
    return state
  else:
    return None

# %% [markdown]
# ## Simulation 1: GHZ State Paradox
#
# Quantum Mechanics predicts correlations for 3 qubits (A, B, C) in the GHZ state:
# Ax * By * Cy = +1
# Ay * Bx * Cy = +1
# Ay * By * Cx = +1
# Ax * Bx * Cx = -1  <-- The problematic one
#
# Multiplying these four equations gives +1 = -1 if we assume outcomes (±1) exist simultaneously and non-contextually. LFT predicts such a simultaneous assignment is logically incoherent and cannot be instantiated.

# %%
# --- GHZ Setup ---
observables = ['Ax', 'Ay', 'Bx', 'By', 'Cx', 'Cy']
outcomes = [-1, 1]

# Define propositions like "P(Ax, 1)" meaning Ax measurement yields +1
def P(obs, val):
    return f"P({obs}, {val})"

# Define GHZ rules as functions checking for violations
# Rule 1: Ax * By * Cy = +1 => Product of values should be +1
def ghz_rule1_violated(state):
    vals = {}
    needed = {'Ax', 'By', 'Cy'}
    for prop in state:
        parts = prop.strip('P()').split(', ')
        if parts[0] in needed:
            vals[parts[0]] = int(parts[1])
    if len(vals) == 3: # Only check if all relevant outcomes are present
        return vals['Ax'] * vals['By'] * vals['Cy'] != 1
    return False # Not enough info in state to violate

def ghz_rule2_violated(state):
    vals = {}
    needed = {'Ay', 'Bx', 'Cy'}
    for prop in state:
        parts = prop.strip('P()').split(', ')
        if parts[0] in needed:
            vals[parts[0]] = int(parts[1])
    if len(vals) == 3:
        return vals['Ay'] * vals['Bx'] * vals['Cy'] != 1
    return False

def ghz_rule3_violated(state):
    vals = {}
    needed = {'Ay', 'By', 'Cx'}
    for prop in state:
        parts = prop.strip('P()').split(', ')
        if parts[0] in needed:
            vals[parts[0]] = int(parts[1])
    if len(vals) == 3:
        return vals['Ay'] * vals['By'] * vals['Cx'] != 1
    return False

# Rule 4 (The contradictory one): Ax * Bx * Cx = -1
def ghz_rule4_violated(state):
    vals = {}
    needed = {'Ax', 'Bx', 'Cx'}
    for prop in state:
        parts = prop.strip('P()').split(', ')
        if parts[0] in needed:
            vals[parts[0]] = int(parts[1])
    if len(vals) == 3:
        return vals['Ax'] * vals['Bx'] * vals['Cx'] != -1 # Check violation
    return False

ghz_rules = [ghz_rule1_violated, ghz_rule2_violated, ghz_rule3_violated, ghz_rule4_violated]

# --- GHZ Simulation ---
# Generate all possible non-contextual assignments of ±1 to the 6 observables
num_possible_assignments = 0
num_coherent_assignments = 0

# Generate combinations of outcomes for the 6 observables
all_assignments = list(itertools.product(outcomes, repeat=len(observables)))
print(f"Total possible assignments to check: {len(all_assignments)}")

coherent_states_found = []

for assignment in all_assignments:
    # Create the potential information state (set of propositions)
    potential_state = set()
    valid_assignment = True
    temp_obs_vals = {}
    for i, obs in enumerate(observables):
        prop = P(obs, assignment[i])
        # Check for immediate contradictions within the assignment logic itself
        # (e.g., cannot assign +1 and -1 to the same observable simultaneously -
        # handled by how we construct the state here)
        if obs in temp_obs_vals and temp_obs_vals[obs] != assignment[i]:
             valid_assignment = False
             break
        temp_obs_vals[obs] = assignment[i]
        potential_state.add(prop)
        # Ensure single value per observable (implicit in itertools.product)
        potential_state.add(negate(P(obs, -assignment[i])))

    if not valid_assignment:
        continue

    num_possible_assignments += 1

    # Apply LFT Projection Operator
    coherent_state = Pi_L(potential_state, ghz_rules)

    if coherent_state is not None:
        num_coherent_assignments += 1
        coherent_states_found.append(coherent_state) # Store if needed

# --- GHZ Results ---
print(f"\n--- GHZ Simulation Results ---")
print(f"Checked {num_possible_assignments} non-contradictory assignments.")
print(f"Number of assignments surviving Π_L (fully coherent with ALL GHZ rules): {num_coherent_assignments}")

if num_coherent_assignments == 0:
    print("LFT Prediction Confirmed: No single information state can simultaneously satisfy all GHZ correlation rules without logical contradiction. Π_L filters out all such potential states.")
else:
    print("LFT Prediction FAILED: Found coherent states satisfying all GHZ rules. This contradicts LFT.")
    # print("Example coherent state:", coherent_states_found[0]) # Optional: print one

# %% [markdown]
# ## Simulation 2: EPR State Correlations
#
# For two particles (A, B) in a singlet state, QM predicts perfect anti-correlation for same-axis spin measurements (e.g., If Az=+1, then Bz=-1). LFT predicts that only states reflecting this logical coherence survive `Π_L`.

# %%
# --- EPR Setup ---
particles = ['A', 'B']
axes = ['z'] # Simulate just one axis for simplicity
outcomes = [-1, 1] # Spin down, Spin up

def P_epr(particle, axis, val):
    return f"P({particle}{axis}, {val})"

# Define EPR rule: Perfect anti-correlation for same-axis measurements
# Rule: ¬(P(Az, v) ∧ P(Bz, v)) for v = +1 or -1
# Rule: (P(Az, v) → P(Bz, -v)) and (P(Bz, v) → P(Az, -v))
def epr_rule_violated(state):
    az_val = None
    bz_val = None
    for prop in state:
        if prop.startswith("P(Az"):
            az_val = int(prop.strip('P()').split(', ')[1])
        elif prop.startswith("P(Bz"):
            bz_val = int(prop.strip('P()').split(', ')[1])

    if az_val is not None and bz_val is not None:
        if az_val == bz_val: # Violation if spins are the same
            # print(f"EPR Violation: Az={az_val}, Bz={bz_val}")
            return True
    return False # Not violated or not enough info

epr_rules = [epr_rule_violated]

# --- EPR Simulation ---
# Generate all 2x2=4 possible outcome combinations for Az and Bz
possible_epr_states = []
for val_a in outcomes:
    for val_b in outcomes:
        state = {
            P_epr('A', 'z', val_a), negate(P_epr('A', 'z', -val_a)),
            P_epr('B', 'z', val_b), negate(P_epr('B', 'z', -val_b))
        }
        possible_epr_states.append(state)

print(f"\n--- EPR Simulation Results ---")
print("Possible joint outcome states:")
coherent_epr_states = []
for i, state in enumerate(possible_epr_states):
    print(f" State {i+1}: {state}")
    coherent_state = Pi_L(state, epr_rules)
    if coherent_state is not None:
        print(f"  -> Coherent under Π_L? Yes")
        coherent_epr_states.append(coherent_state)
    else:
        print(f"  -> Coherent under Π_L? No (Filtered Out)")

print(f"\nTotal Coherent EPR states found: {len(coherent_epr_states)}")
print("Coherent states:")
for state in coherent_epr_states:
    print(f" - {state}")

print("\nLFT Prediction: Only states showing perfect anti-correlation (Az=+1, Bz=-1 or Az=-1, Bz=+1) survive Π_L, matching experimental observation.")

# %% [markdown]
# ## Conclusion
#
# These simulations illustrate LFT's core filtering mechanism (`Π_L`).
# - For GHZ, the inherent logical contradiction in assigning non-contextual values satisfying all QM predictions simultaneously leads to `Π_L` rejecting *all* such complete assignments.
# - For EPR, `Π_L` enforces the observed anti-correlations by rejecting states where both particles have the same spin outcome along the same axis.
#
# This shows LFT's potential to explain *why* certain configurations are forbidden (logical impossibility) rather than just stating correlation rules.

Total possible assignments to check: 64

--- GHZ Simulation Results ---
Checked 64 non-contradictory assignments.
Number of assignments surviving Π_L (fully coherent with ALL GHZ rules): 0
LFT Prediction Confirmed: No single information state can simultaneously satisfy all GHZ correlation rules without logical contradiction. Π_L filters out all such potential states.

--- EPR Simulation Results ---
Possible joint outcome states:
 State 1: {'P(Bz, -1)', '¬P(Bz, 1)', '¬P(Az, 1)', 'P(Az, -1)'}
  -> Coherent under Π_L? No (Filtered Out)
 State 2: {'P(Bz, 1)', '¬P(Bz, -1)', '¬P(Az, 1)', 'P(Az, -1)'}
  -> Coherent under Π_L? Yes
 State 3: {'P(Bz, -1)', '¬P(Az, -1)', '¬P(Bz, 1)', 'P(Az, 1)'}
  -> Coherent under Π_L? Yes
 State 4: {'P(Bz, 1)', '¬P(Az, -1)', '¬P(Bz, -1)', 'P(Az, 1)'}
  -> Coherent under Π_L? No (Filtered Out)

Total Coherent EPR states found: 2
Coherent states:
 - {'P(Bz, 1)', '¬P(Bz, -1)', '¬P(Az, 1)', 'P(Az, -1)'}
 - {'P(Bz, -1)', '¬P(Az, -1)', '¬P(Bz, 1)', 'P(Az, 1)'}

LFT P